<a href="https://colab.research.google.com/github/teteumac/DileptonAnalysis/blob/master/pps_dilepton_ml_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
!python3 -m pip install --user uproot4 awkward1 mplhep lightgbm

In [23]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sys
import seaborn as sns
from lightgbm import LGBMClassifier
import lightgbm as lgb
import sys

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import PrecisionRecallDisplay,precision_score,plot_roc_curve,recall_score,accuracy_score,log_loss,roc_auc_score,classification_report,f1_score,confusion_matrix,roc_curve,precision_recall_curve,plot_precision_recall_curve,average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


In [24]:
%matplotlib inline

In [25]:
proton_selection = "MultiRP"

df_signal = None
event_counts_signal = None
selections_str_signal = None


# SIGNAL

In [26]:
fileName_signal_exclusive = '/content/gdrive/My Drive/pps_ML/output-Exclusive130.h5'


with h5py.File( fileName_signal_exclusive, 'r' ) as f:
    print ( list(f.keys()) )
    dset = f['protons']
    print ( dset.shape )
    print ( dset[:,:] )
    dset_columns = f['columns']
    print ( dset_columns.shape )
    columns = list( dset_columns )
    print ( columns )
    columns_str = [ item.decode("utf-8") for item in columns ]
    print ( columns_str )
    
    dset_counts = f['event_counts']
    event_counts_signal = list( dset_counts )
    print ( event_counts_signal )
    
    dset_selections = f['selections']
    selections_ = list( dset_selections )
    print ( selections_ )
    selections_str_signal = [ item.decode("utf-8") for item in selections_ ]
    print ( selections_str_signal )
    
    df_signal = pd.DataFrame( dset, columns=columns_str )
    
    #df_signal = df_signal[ ['Xi', 'T', 'ThX', 'ThY', 'Weight' , "pt", "dz1", "XiMuMuPlus", "MultiRP",
    #                  'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuMinus', 'PrimVertexZ'] ]

df_counts_signal_exclusive = pd.Series( event_counts_signal, index=selections_str_signal );
#print( df_counts_signal_exclusive )
msk_sig = (df_signal["InvMass"] >= 110. )

msk1_sig = None
msk2_sig = None
if proton_selection == "SingleRP":
    # Single-RP in pixel stations
    msk1_arm = ( df_signal["RPId1"] == 23 )
    msk2_arm = ( df_signal["RPId1"] == 123 )
    df_signal[ "XiMuMu" ] = np.nan
    df_signal[ "XiMuMu" ].where( ~msk1_arm, df_signal[ "XiMuMuPlus" ], inplace=True )
    df_signal[ "XiMuMu" ].where( ~msk2_arm, df_signal[ "XiMuMuMinus" ], inplace=True )
    #df_signal[ "XiMuMu" ][ msk2_arm ] = df_signal[ "XiMuMuMinus" ] 
    msk1_sig = msk_sig & ( df_signal["MultiRP"] == 0) & msk1_arm
    msk2_sig = msk_sig & ( df_signal["MultiRP"] == 0) & msk2_arm
elif proton_selection == "MultiRP":
    # Multi-RP
    msk1_arm = ( df_signal["Arm"] == 0  )
    msk2_arm = ( df_signal["Arm"] == 1  )
    df_signal[ "XiMuMu" ] = np.nan
    df_signal[ "XiMuMu" ].where( ~msk1_arm, df_signal[ "XiMuMuPlus" ], inplace=True )
    df_signal[ "XiMuMu" ].where( ~msk2_arm, df_signal[ "XiMuMuMinus" ], inplace=True )
    msk1_sig = msk_sig & ( df_signal["MultiRP"] == 1 ) & msk1_arm 
    msk2_sig = msk_sig & ( df_signal["MultiRP"] == 1 ) & msk2_arm

df_signal = df_signal[ msk1_sig | msk2_sig ].dropna()
df_signal['target'] = 1
print( df_signal )

['columns', 'event_counts', 'protons', 'selections']
(285346, 15)
[[ 3.83796021e-02 -1.04100592e-02  0.00000000e+00 ...  2.31398208e-03
   1.09153595e-02  2.14630890e+00]
 [ 3.81265022e-02 -1.48317078e-02  0.00000000e+00 ...  2.31398208e-03
   1.09153595e-02  2.14630890e+00]
 [ 3.94197255e-02 -4.52564545e-02 -1.39372623e-05 ...  2.31398208e-03
   1.09153595e-02  2.14630890e+00]
 ...
 [ 4.01983038e-02 -3.09351496e-02  0.00000000e+00 ...  1.85212595e-04
   3.05966531e-03 -5.42296886e+00]
 [ 3.63693014e-02 -1.42346010e-01  4.58586255e-05 ...  1.85212595e-04
   3.05966531e-03 -5.42296886e+00]
 [ 2.03184187e-02 -4.13834184e-01  0.00000000e+00 ...  1.50995244e-04
   4.87857127e-03  1.37020195e+00]]
(15,)
[b'Xi', b'T', b'ThX', b'ThY', b'Weight', b'pt', b'dz1', b'XiMuMuPlus', b'MultiRP', b'Arm', b'InvMass', b'ExtraPfCands', b'Acopl', b'XiMuMuMinus', b'PrimVertexZ']
['Xi', 'T', 'ThX', 'ThY', 'Weight', 'pt', 'dz1', 'XiMuMuPlus', 'MultiRP', 'Arm', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuMinus'

In [27]:
df_signal

,Xi,T,ThX,ThY,Weight,pt,dz1,XiMuMuPlus,MultiRP,Arm,InvMass,ExtraPfCands,Acopl,XiMuMuMinus,PrimVertexZ,XiMuMu,target
2,0.039420,-0.045256,-0.000014,0.000030,1.208416,2.305948,0.004524,0.039576,1.0,0.0,270.187437,2.0,0.002314,0.010915,2.146309,0.039576,1
5,0.038857,-0.233490,-0.000031,0.000069,1.154366,1.486977,0.000970,0.037688,1.0,0.0,209.140113,1.0,0.000431,0.006868,0.247230,0.037688,1
8,0.045908,-0.294082,0.000085,-0.000001,1.009501,0.662313,0.006846,0.003999,1.0,1.0,177.482574,0.0,0.000732,0.046605,0.633745,0.046605,1
11,0.051476,-0.074141,-0.000041,0.000012,1.154366,0.219762,0.002116,0.050538,1.0,0.0,116.882889,1.0,0.000277,0.001600,2.956322,0.050538,1
16,0.040869,-0.038973,0.000029,-0.000010,1.241477,0.222753,0.002141,0.038771,1.0,0.0,139.021805,0.0,0.000297,0.002950,-0.174605,0.038771,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285330,0.030848,-0.041254,0.000031,-0.000005,1.251805,3.785275,0.002101,0.012214,1.0,1.0,251.145794,11.0,0.002766,0.030564,3.360268,0.030564,1
285335,0.035900,-0.360529,0.000017,-0.000092,0.860627,2.222014,0.000058,0.007713,1.0,1.0,218.789124,3.0,0.000580,0.036727,2.482682,0.036727,1
285338,0.051403,-0.494220,0.000070,-0.000086,0.795806,5.244809,0.000192,0.011024,1.0,1.0,301.041639,0.0,0.000083,0.048657,-0.382731,0.048657,1
285341,0.042523,-0.064319,-0.000025,-0.000031,1.518433,0.868856,0.006715,0.002110,1.0,1.0,127.194376,3.0,0.000267,0.045377,0.996049,0.045377,1


In [28]:

fileName_signal_semiexclusive = '/content/gdrive/My Drive/pps_ML/output-SemiExclusive130.h5'

with h5py.File( fileName_signal_semiexclusive, 'r' ) as f:
    print ( list(f.keys()) )
    dset = f['protons']
    print ( dset.shape )
    print ( dset[:,:] )
    dset_columns = f['columns']
    print ( dset_columns.shape )
    columns = list( dset_columns )
    print ( columns )
    columns_str = [ item.decode("utf-8") for item in columns ]
    print ( columns_str )
    
    dset_counts = f['event_counts']
    event_counts_signal = list( dset_counts )
    print ( event_counts_signal )
    
    dset_selections = f['selections']
    selections_ = list( dset_selections )
    print ( selections_ )
    selections_str_signal = [ item.decode("utf-8") for item in selections_ ]
    print ( selections_str_signal )
    
    df_signal_semi = pd.DataFrame( dset, columns=columns_str )                      

df_counts_signal_semiexclusive = pd.Series( event_counts_signal, index=selections_str_signal );
print( df_counts_signal_semiexclusive )

msk_signal_semi = (df_signal_semi["InvMass"] >= 0)

msk1_signal_semi = None
msk2_signal_semi = None
if proton_selection == "SingleRP":
    # Single-RP in pixel stations
    msk1_arm = ( df_signal_semi["RPId1"] == 23 )
    msk2_arm = ( df_signal_semi["RPId1"] == 123 )
    df_signal_semi[ "XiMuMu" ] = np.nan
    df_signal_semi[ "XiMuMu" ].where( ~msk1_arm, df_signal_semi[ "XiMuMuPlus" ], inplace=True )
    df_signal_semi[ "XiMuMu" ].where( ~msk2_arm, df_signal_semi[ "XiMuMuMinus" ], inplace=True )
    #df_signal[ "XiMuMu" ][ msk2_arm ] = df_signal[ "XiMuMuMinus" ] 
    msk1_sig = msk_sig & ( df_signal_semi["MultiRP"] == 0) & msk1_arm
    msk2_sig = msk_sig & ( df_signal_semi["MultiRP"] == 0) & msk2_arm
elif proton_selection == "MultiRP":
    # Multi-RP
    msk1_arm = ( df_signal_semi["Arm"] == 0 )
    msk2_arm = ( df_signal_semi["Arm"] == 1 )
    df_signal_semi[ "XiMuMu" ] = np.nan
    df_signal_semi[ "XiMuMu" ].where( ~msk1_arm, df_signal_semi[ "XiMuMuPlus" ], inplace=True )
    df_signal_semi[ "XiMuMu" ].where( ~msk2_arm, df_signal_semi[ "XiMuMuMinus" ], inplace=True )

    msk1_signal_semi = msk_signal_semi & ( df_signal_semi["MultiRP"] == 1 ) & msk1_arm
    msk2_signal_semi = msk_signal_semi & ( df_signal_semi["MultiRP"] == 1 ) & msk2_arm

df_signal_semi = df_signal_semi[ msk1_signal_semi | msk2_signal_semi ]
df_signal_semi = df_signal_semi.dropna()
df_signal_semi['target'] = 2
print( df_signal_semi )

['columns', 'event_counts', 'protons', 'selections']
(163308, 15)
[[ 3.63221094e-02 -1.15129407e-02  0.00000000e+00 ...  8.10643436e-03
   3.94431096e-02  9.19979155e-01]
 [ 3.58829275e-02 -1.85621530e-02  0.00000000e+00 ...  8.10643436e-03
   3.94431096e-02  9.19979155e-01]
 [ 3.79030667e-02 -5.86915016e-02 -1.58526582e-05 ...  8.10643436e-03
   3.94431096e-02  9.19979155e-01]
 ...
 [ 7.24671632e-02 -3.74770537e-02  0.00000000e+00 ...  1.93572367e-03
   3.88485338e-03  1.68140030e+00]
 [ 7.22354800e-02 -4.11085933e-02  0.00000000e+00 ...  1.93572367e-03
   3.88485338e-03  1.68140030e+00]
 [ 7.44942874e-02 -1.22550160e-01 -1.23130258e-05 ...  1.93572367e-03
   3.88485338e-03  1.68140030e+00]]
(15,)
[b'Xi', b'T', b'ThX', b'ThY', b'Weight', b'pt', b'dz1', b'XiMuMuPlus', b'MultiRP', b'Arm', b'InvMass', b'ExtraPfCands', b'Acopl', b'XiMuMuMinus', b'PrimVertexZ']
['Xi', 'T', 'ThX', 'ThY', 'Weight', 'pt', 'dz1', 'XiMuMuPlus', 'MultiRP', 'Arm', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuMinus'

In [29]:
df_signal_semi

,Xi,T,ThX,ThY,Weight,pt,dz1,XiMuMuPlus,MultiRP,Arm,InvMass,ExtraPfCands,Acopl,XiMuMuMinus,PrimVertexZ,XiMuMu,target
2,0.037903,-0.058692,-0.000016,0.000034,1.069333,3.308203,0.000656,0.007522,1.0,1.0,223.903747,0.0,0.008106,0.039443,0.919979,0.039443,2
5,0.029967,-0.619747,0.000104,-0.000065,1.212219,1.036125,0.000865,0.030873,1.0,0.0,151.804255,1.0,0.004339,0.004417,3.731446,0.030873,2
8,0.056337,-0.042861,0.000028,-0.000014,1.259455,4.853525,0.001198,0.060257,1.0,0.0,515.383351,1.0,0.011116,0.026086,4.791252,0.060257,2
11,0.054751,-0.012992,0.000004,0.000016,1.092546,157.975805,0.000267,0.010878,1.0,1.0,264.764781,1.0,0.152455,0.051705,-3.785355,0.051705,2
14,0.085117,-0.069233,0.000014,0.000038,1.212219,37.766873,0.000407,0.010340,1.0,1.0,374.321953,1.0,0.062509,0.081000,-0.906035,0.081000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163291,0.037883,-0.147633,0.000050,-0.000033,1.561522,16.232387,0.001012,0.037889,1.0,0.0,157.571687,4.0,0.010142,0.003919,0.540850,0.037889,2
163294,0.045255,-0.029653,0.000003,-0.000026,0.973364,1.252627,0.005973,0.001716,1.0,1.0,116.170305,0.0,0.001037,0.046531,-0.896035,0.046531,2
163299,0.061012,-0.178772,-0.000060,0.000030,1.182239,4.131175,0.004376,0.034717,1.0,1.0,598.238735,0.0,0.004072,0.061002,-2.230065,0.061002,2
163304,0.043050,-0.054138,0.000032,-0.000017,1.154672,139.103603,0.000232,0.005914,1.0,1.0,160.153401,0.0,0.472806,0.045022,3.994493,0.045022,2


# Background

In [ ]:
df_bkg_list = []
df_counts_bkg_list = []

resample_factor = 20

fileNames_bkg = [
'/content/gdrive/My Drive/pps_ML/output-DatadrivenB.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenC0.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenC1.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenC2.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenC3.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenE0.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenE1.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenE2.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenE3.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenE4.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenE5.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenE6.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenF0.h5',
'/content/gdrive/My Drive/pps_ML/output-DatadrivenF1.h5'
]

for file_ in fileNames_bkg:
    with h5py.File( file_, 'r' ) as f:
        #print ( file_ )    
        #print ( list(f.keys()) )
        dset = f['protons']
        #print ( dset.shape )
        #print ( dset[:,:] )
        
        dset_columns = f['columns']
        #print ( dset_columns.shape )
        columns = list( dset_columns )
        #print ( columns )
        columns_str = [ item.decode("utf-8") for item in columns ]
        #print ( columns_str )

        dset_selections = f['selections']
        selections_ = [ item.decode("utf-8") for item in dset_selections ]
        #print ( selections_ )
        
        dset_counts = f['event_counts']
        df_counts_bkg_list.append( pd.Series( dset_counts, index=selections_ ) )
        #print ( df_counts_bkg_list[-1] )
       
        chunk_size = 1000000
        entries = dset.shape[0]
        start_ = list( range( 0, entries, chunk_size ) )
        stop_ = start_[1:]
        stop_.append( entries )
        #print ( start_ )
        #print ( stop_ )
        for idx in range( len( start_ ) ):
            #print ( start_[idx], stop_[idx] )
            #print ( dset[ start_[idx] : stop_[idx] ] )
            df_ = pd.DataFrame( dset[ start_[idx] : stop_[idx] ], columns=columns_str )
            #df_ = df_[ ['Run', 'LumiSection', 'EventNum', 'CrossingAngle', 
            #            'MultiRP', 'Arm', 'RPId1', 'RPId2', 'TrackX1', 'TrackY1', 'TrackX2', 'TrackY2',
            #            'Xi', 'T', 'ThX', 'ThY', 'Time', 'Weight',
            #            'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuPlus', 'XiMuMuMinus'] ].astype( { "Run": "int64", "LumiSection": "int64", "EventNum": "int64", "MultiRP": "int32", "Arm": "int32", "RPId1": "int32", "RPId2": "int32", "ExtraPfCands": "int32" } )
            df_bkg_list.append( df_ )
            #print ( df_bkg_list[-1].head() )
            #print ( len( df_bkg_list[-1] ) )

df_counts_bkg = df_counts_bkg_list[0]
for idx in range( 1, len( df_counts_bkg_list ) ):
    df_counts_bkg = df_counts_bkg.add( df_counts_bkg_list[idx] )
print( df_counts_bkg )            

df_bkg = pd.concat( df_bkg_list, ignore_index=True )

msk_bkg = ( df_bkg["InvMass"] >= 0. )

msk1_bkg = None
msk2_bkg = None
if proton_selection == "SingleRP":
    # Single-RP in pixel stations
    msk1_arm = ( df_bkg["RPId1"] == 23 )
    msk2_arm = ( df_bkg["RPId1"] == 123 )
    df_bkg[ "XiMuMu" ] = np.nan
    df_bkg[ "XiMuMu" ].where( ~msk1_arm, df_bkg[ "XiMuMuPlus" ], inplace=True )
    df_bkg[ "XiMuMu" ].where( ~msk2_arm, df_bkg[ "XiMuMuMinus" ], inplace=True )
    msk1_bkg = msk_bkg & ( df_bkg["MultiRP"] == 0) & msk1_arm
    msk2_bkg = msk_bkg & ( df_bkg["MultiRP"] == 0) & msk2_arm
elif proton_selection == "MultiRP":
    # Multi-RP
    msk1_arm = ( df_bkg["Arm"] == 0 )
    msk2_arm = ( df_bkg["Arm"] == 1 )
    df_bkg[ "XiMuMu" ] = np.nan
    df_bkg[ "XiMuMu" ].where( ~msk1_arm, df_bkg[ "XiMuMuPlus" ], inplace=True )
    df_bkg[ "XiMuMu" ].where( ~msk2_arm, df_bkg[ "XiMuMuMinus" ], inplace=True )
    #df_bkg[ "Xi" ].where( ~msk1_arm, df_bkg['Xi'][df_bkg['Arm']==0].loc[df_bkg['Xi'][df_bkg['Arm']==0]>0.045], inplace=True )
    #df_bkg[ "Xi" ].where( ~msk2_arm, df_bkg['Xi'][df_bkg['Arm']==1].loc[df_bkg['Xi'][df_bkg['Arm']==1]>0.055], inplace=True )
    msk1_bkg = msk_bkg & ( df_bkg["MultiRP"] == 1 ) & msk1_arm
    msk2_bkg = msk_bkg & ( df_bkg["MultiRP"] == 1 ) & msk2_arm

df_bkg = df_bkg[ msk1_bkg | msk2_bkg ]
df_bkg['target'] = 0 

All           834616760
Muon            8130540
Vertex          8130540
ProtonsRnd      6573240
dtype: int64


In [ ]:
df_bkg

# NORMALIZAÇÃO E CONTAGEM DE EVENTOS

In [ ]:
n_events_h_elastic = 200000
n_events_h_inel_el = 200000
n_events_h_dy = 48675378.
total_lumi = 37.498268
limit_lumi = 18.346268
rap_gap_surv_elastic = 0.89
rap_gap_surv_inel_el = 0.76
rap_gap_surv_inelastic = 0.13
number_of_samples = 20

scale_factor_elastic_x130 = limit_lumi*rap_gap_surv_elastic*0.017254036*1000 / n_events_h_elastic
scale_factor_inel_el_x130 = limit_lumi*rap_gap_surv_inel_el*0.025643500*1000 / n_events_h_inel_el 
scale_factor_dy = limit_lumi*5334000 / n_events_h_dy
scale_factor_dy_resample = limit_lumi*5334000 / ( n_events_h_dy*number_of_samples )
scale_factor_pseudoback = 1/number_of_samples

df_signal.loc[:,'Weight'] *= scale_factor_elastic_x130
df_signal_semi.loc[:,'Weight'] *= scale_factor_inel_el_x130
df_bkg.loc[:,'Weight'] *= scale_factor_pseudoback

DataSet2 = pd.concat([df_signal,df_signal_semi,df_bkg], ignore_index = True)

In [ ]:
drop_columns = ['target', 'Weight', "MultiRP", "Arm", 'XiMuMuPlus','XiMuMuMinus']
# Elemina as colunas que desejar

In [ ]:
DataSet2 = DataSet2[ ((DataSet2['Arm']==1) & (DataSet2['Xi'] > 0.055)) | ((DataSet2['Arm']==0) & (DataSet2['Xi'] > 0.045))] # selecionando os Xi

In [ ]:
colunas_ = DataSet2.columns

In [ ]:
a = pd.DataFrame(np.array(DataSet2), columns = colunas_).drop(['Weight', "MultiRP", "Arm", 'XiMuMuPlus','XiMuMuMinus'],axis=1)

In [ ]:
fig= plt.figure( figsize=(10,10) )
label_conts = a['target'].replace([0,1,2],['Data-driven Background','Exclusivo','Semi-Exclusivo'])
sns.set(style="darkgrid")
sns.countplot(x=label_conts, data=a)
#plt.style.use(hep.style.CMS)
plt.title('Contagem do número de eventos para cada classe')
plt.yscale('log')
plt.tight_layout()

In [ ]:
DataSet2[DataSet2['target']==2].Weight.sum()

In [ ]:
import mplhep as hep

# PROCURA POR HIPERPARÂMETROS E TREINAMENTO DO MODELO

In [ ]:

DataSet_Train, dataset_test = train_test_split( DataSet2, test_size = 0.50, random_state = 41 )
'''
with h5py.File( 'amostra_train' + '.h5', 'w') as f:
    dset = f.create_dataset( 'train', data=DataSet_Train )

with h5py.File( 'amostra_test' + '.h5', 'w') as f:
    dset = f.create_dataset( 'test', data=dataset_test )    
'''
Norm_Test = 1/0.50

y_train_ = DataSet_Train['target'].replace(2,1) # Muda os valores de 2 para 1
y_test_ = dataset_test['target'].replace(2,1)

train_data = lgb.Dataset(DataSet_Train.drop( drop_columns , axis = 1 ), label = y_train_ )
test_data = lgb.Dataset(dataset_test.drop( drop_columns , axis = 1 ) , label = y_test_ )

n_iter = 4
cv = 3
scoring = [ 'roc_auc_score','f1_score','precision_score','recall_score' ]

param_grid = {
    'learning_rate': list(np.arange(0.001,0.01,0.001)), 
    'n_estimators': list(range(700, 1000, 2)),
    'num_leaves': list(range(60, 100, 2)),
    'max_depth': list(range(0, 42, 2)),
    'min_child_samples': list(range(0, 30, 1)),
    'subsample': list(np.arange(0.3,0.55,0.01)), 
    'colsample_bytree':list(np.arange(0.5,1,0.01))
              }  

mdl = LGBMClassifier( boosting_type = 'gbdt' , objective = 'binary' )


search = RandomizedSearchCV( estimator = mdl, param_distributions = param_grid, n_iter = n_iter, refit=scoring ,cv = cv, verbose = 1 )

search_result = search.fit(DataSet_Train.drop( drop_columns , axis = 1), y_train_)

# Results
print("Best: %f using %s" % ( search_result.best_score_, search_result.best_params_ ) )
print('\n')

In [ ]:
#sns.pairplot(DataSet_Train.drop(['target', 'Weight', "MultiRP","Arm"], axis = 1))

In [ ]:
DataSet_Train

In [ ]:
dataset_test

In [ ]:
predict_proba_LGBM = search_result.predict_proba( dataset_test.drop( drop_columns , axis = 1 ) )[:,1] 

fpr_lgb, tpr_lgb, thresholds_lgb = roc_curve( y_test_, predict_proba_LGBM ) # fpr -> false positive rate | tpr -> true positive rate
prec_lgb, rec_lgb, threshs_lgb = precision_recall_curve( y_test_, predict_proba_LGBM ) # prec -> precision | rec -> recall 
f1 = 2 * ( prec_lgb * rec_lgb ) / ( prec_lgb + rec_lgb )
threshs_lgb  = np.concatenate( [ threshs_lgb , [1] ] , axis = 0 )

bidx = np.argmax(prec_lgb*rec_lgb)
best_cut = threshs_lgb[bidx]
print('\n',' Best Cut ------>> ', best_cut,'\n')
y_pred_cut = predict_proba_LGBM >= best_cut

In [ ]:
from pandas import DataFrame

fig= plt.figure( figsize=(10,10) )
conf_mat = confusion_matrix(y_true=y_test_, y_pred=y_pred_cut)
print('Confusion matrix:\n', conf_mat)
conf_mat = DataFrame(conf_mat)
conf_mat = conf_mat.rename(columns={0:'Background', 1:'Signal'})
conf_mat = conf_mat.T
conf_mat = conf_mat.rename(columns={0:'Background', 1:'Signal'})
conf_mat = conf_mat.T
sns.heatmap(conf_mat, annot=True, fmt="d",cmap = 'RdBu')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/confusion_matrix.png')
plt.close()


In [ ]:
plt.figure( figsize=(10,10) )
plot_precision_recall_curve( search, dataset_test.drop( drop_columns ,axis=1),y_test_, color = 'red')
#plt.style.use(hep.style.CMS)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.show()
#plt.savefig('/home/matheus/test/pre_rec.png')
plt.close()

In [ ]:
fig= plt.figure( figsize=(10,10) )
plot_roc_curve( search, dataset_test.drop( drop_columns ,axis=1),y_test_, color = 'red')
plt.title("ROC Curve")
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
#plt.style.use(hep.style.CMS)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.show()
#plt.savefig('/home/matheus/test/roc.png')
plt.close()


In [ ]:
fig= plt.figure( figsize=(10,10) )
plt.plot( threshs_lgb , prec_lgb*rec_lgb ,  color = 'purple' )
plt.plot( [ best_cut,best_cut ] , [0,(prec_lgb * rec_lgb).max()*2] ,"-.r",label='Best Cut : {:2.5f}'.format(best_cut) )
plt.plot( [-0.1,1.1] , [ (prec_lgb*rec_lgb).max(),(prec_lgb*rec_lgb).max()], "--g",label='precision*recall : {:2.5f}'.format((prec_lgb*rec_lgb).max()) )
plt.ylabel( 'PRECISION * RECALL' )
plt.xlabel(    'THRESHOLDS'    )
plt.ylim( -0.002 , 1.4 )
plt.xlim( -0.01 , 1.05)
plt.legend(loc="best")
#plt.style.use(hep.style.CMS)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.show()
#plt.savefig('/home/matheus/test/prec_x_rec.png')
plt.close()

In [ ]:
fig= plt.figure( figsize=(12,10) )
hbgt_lgb =  plt.hist(predict_proba_LGBM[y_test_==0],bins=np.linspace(0,1,50), histtype='step',label='Background', color = 'b')
hsigt_lgb = plt.hist(predict_proba_LGBM[y_test_==1],bins=np.linspace(0,1,50), histtype='step',label='Signal', color = 'orange')
uppery_lgb=np.max(hbgt_lgb[0])
plt.plot([best_cut,best_cut],[0,uppery_lgb],"-.r",label='Best cut : {:2.2f}'.format(best_cut))
plt.xlabel("Probability")
plt.ylabel("Events")
plt.legend(loc="upper right")
plt.text(0.1,50e4, "Purity: {:2.2f}%".format(100*precision_score(y_test_,y_pred_cut)),fontsize = 18)
plt.text(0.1,10e4, "Efficiency: {:2.2f}%".format(100*recall_score(y_test_,y_pred_cut)),fontsize = 18)
#plt.text(0.1,10e3, "Accuracy: {:2.2f}%".format(100*accuracy_score(y_test_,y_pred_cut)), fontsize = 18)
plt.text(0.1,10e3, "ROC AUC: {:2.2f}%".format(100*roc_auc_score(y_test_,y_pred_cut)), fontsize = 18)
plt.text(0.1,10e2, "F1_Score: {:2.2f}%".format(100*f1_score(y_test_,y_pred_cut)), fontsize = 18)
plt.text(best_cut+0.06,1000, 'SIGNAL REGION', color = 'red', fontsize = 25)
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,3), useMathText = True)
#plt.style.use(hep.style.CMS)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.yscale('log')
plt.tight_layout()
plt.figure(figsize = (20, 12))
plt.show()
#plt.savefig('/home/matheus/test/discriminante.png')
plt.close()

## **DADOS**

In [ ]:
df_data_list = []
df_counts_data_list = []

fileNames = [
'/content/gdrive/My Drive/pps_ML/output-DataB.h5',
'/content/gdrive/My Drive/pps_ML/output-DataC0.h5',
'/content/gdrive/My Drive/pps_ML/output-DataC1.h5',
'/content/gdrive/My Drive/pps_ML/output-DataC2.h5',
'/content/gdrive/My Drive/pps_ML/output-DataC3.h5',
'/content/gdrive/My Drive/pps_ML/output-DataE0.h5',
'/content/gdrive/My Drive/pps_ML/output-DataE1.h5',
'/content/gdrive/My Drive/pps_ML/output-DataE2.h5',
'/content/gdrive/My Drive/pps_ML/output-DataE3.h5',
'/content/gdrive/My Drive/pps_ML/output-DataE4.h5',
'/content/gdrive/My Drive/pps_ML/output-DataE5.h5',
'/content/gdrive/My Drive/pps_ML/output-DataE6.h5',
'/content/gdrive/My Drive/pps_ML/output-DataF0.h5',
'/content/gdrive/My Drive/pps_ML/output-DataF1.h5'
]

In [ ]:
for file_ in fileNames:
    with h5py.File( file_, 'r' ) as f:
        print ( f )
        print ( list(f.keys()) )
        dset = f['protons']
        print ( dset.shape )
        print ( dset[:,:] )
        
        dset_columns = f['columns']
        print ( dset_columns.shape )
        columns = list( dset_columns )
        print ( columns )
        columns_str = [ item.decode("utf-8") for item in columns ]
        print ( columns_str )

        dset_selections = f['selections']
        selections_ = [ item.decode("utf-8") for item in dset_selections ]
        print ( selections_ )  
        
        dset_counts = f['event_counts']
        df_counts_data_list.append( pd.Series( dset_counts, index=selections_ ) )
        print ( df_counts_data_list[-1] )
        
        chunk_size = 1000000
        entries = dset.shape[0]
        start_ = list( range( 0, entries, chunk_size ) )
        stop_ = start_[1:]
        stop_.append( entries )
        print ( start_ )
        print ( stop_ )
        for idx in range( len( start_ ) ):
            print ( start_[idx], stop_[idx] )
            #print ( dset[ start_[idx] : stop_[idx] ] )
            df_ = pd.DataFrame( dset[ start_[idx] : stop_[idx] ], columns=columns_str )
            df_data_list.append( df_ )
            print ( df_data_list[-1].head() )
            print ( len( df_data_list[-1] ) )

In [ ]:
df_counts_data = df_counts_data_list[0];
for idx in range( 1, len(df_counts_data_list) ):
    df_counts_data = df_counts_data.add( df_counts_data_list[idx] )
df_counts_data


In [ ]:
df_data = pd.concat( df_data_list )
print( df_data )
print( df_data.columns )

In [ ]:
msk_data = ( df_data["InvMass"] >= 0. )

msk1_data = None
msk2_data = None
if proton_selection == "SingleRP":
    # Single-RP in pixel stations
    msk1_arm = ( df_data["RPId1"] == 23 )
    msk2_arm = ( df_data["RPId1"] == 123 )
    df_data[ "XiMuMu" ] = np.nan
    df_data[ "XiMuMu" ].where( ~msk1_arm, df_data[ "XiMuMuPlus" ], inplace=True )
    df_data[ "XiMuMu" ].where( ~msk2_arm, df_data[ "XiMuMuMinus" ], inplace=True )
    msk1_data = msk_data & ( df_data["MultiRP"] == 0) & msk1_arm
    msk2_data = msk_data & ( df_data["MultiRP"] == 0) & msk2_arm
elif proton_selection == "MultiRP":
    # Multi-RP
    msk1_arm = ( df_data["Arm"] == 0 )
    msk2_arm = ( df_data["Arm"] == 1 )
    df_data[ "XiMuMu" ] = np.nan
    df_data[ "XiMuMu" ].where( ~msk1_arm, df_data[ "XiMuMuPlus" ], inplace=True )
    df_data[ "XiMuMu" ].where( ~msk2_arm, df_data[ "XiMuMuMinus" ], inplace=True )
    msk1_data = msk_data & ( df_data["MultiRP"] == 1 ) & msk1_arm
    msk2_data = msk_data & ( df_data["MultiRP"] == 1 ) & msk2_arm

df_data = df_data[ msk1_data | msk2_data ]
print( df_data )
print( df_data.columns )

In [ ]:
df_data = df_data[ ((df_data['Arm']==1) & (df_data['Xi'] > 0.055)) | ((df_data['Arm']==0) & (df_data['Xi'] > 0.045))] # selecionando os Xi

In [ ]:
df_data

In [ ]:
MonteCarlo_DrellYan = DataFrame( np.array( dataset_test[ dataset_test.target == 0 ]  ) , columns = DataSet_Train.columns)
MonteCarlo_Exclusive = DataFrame( np.array( dataset_test[ dataset_test.target == 1 ]  ) , columns = DataSet_Train.columns )
MonteCarlo_SemiExclusive = DataFrame( np.array( dataset_test[ dataset_test.target == 2 ] ) , columns = DataSet_Train.columns)
#print(MonteCarlo_SemiExclusive)
#print(MonteCarlo_Exclusive)
#print(MonteCarlo_DrellYan)

In [ ]:
predict_proba_LGBM_background = search_result.predict_proba( MonteCarlo_DrellYan.drop( drop_columns , axis = 1 ) )[:,1]
predict_proba_LGBM_exclusivo = search_result.predict_proba( MonteCarlo_Exclusive.drop( drop_columns , axis = 1 ) )[:,1]
predict_proba_LGBM_semiexclusivo = search_result.predict_proba( MonteCarlo_SemiExclusive.drop( drop_columns , axis = 1 ) )[:,1]
predict_proba_LGBM_Data = search_result.predict_proba( df_data.drop( [ 'MultiRP','Arm', 'Weight', 'XiMuMuPlus', 'XiMuMuMinus'] , axis = 1 ) )[:,1]

In [ ]:
class_true = dataset_test['target'].replace(2,1)

prec, rec, threshs = precision_recall_curve( y_test_, predict_proba_LGBM , sample_weight = dataset_test.Weight*Norm_Test ) # prec -> precision | rec -> recall 
fpr, tpr, thresholds = roc_curve( y_test_, predict_proba_LGBM , sample_weight = dataset_test.Weight*Norm_Test )
f1 = 2 * ( prec * rec ) / ( prec + rec )
threshs  = np.concatenate( [ threshs , [1] ] , axis = 0 )

argmax = np.argmax( prec * rec )
best_cut = threshs[argmax]
print('\n',' Best Cut ------>> ', best_cut,'\n')
y_pred_cut = predict_proba_LGBM >= best_cut
y_pred_cut = np.array(y_pred_cut)

In [ ]:
fig= plt.figure( figsize=(10,10) )
plot_precision_recall_curve( search, dataset_test.drop( drop_columns ,axis=1),y_test_, color = 'red',sample_weight = dataset_test.Weight*Norm_Test)
#plt.style.use(hep.style.CMS)
plt.plot([0, 1], [0,0], color='black', lw=2, linestyle='--')
plt.legend(loc="best")
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.show()
#plt.savefig('/home/matheus/test/prec_rec_normalized.png')
plt.close()

In [ ]:
plot_roc_curve( search, dataset_test.drop( drop_columns ,axis=1),y_test_, color = 'red',sample_weight = dataset_test.Weight*Norm_Test)
plt.title("ROC Curve")
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
#plt.style.use(hep.style.CMS)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.show()
#plt.savefig('/home/matheus/test/roc_normalized.png')
plt.close()

In [ ]:
fig= plt.figure( figsize=(10,10) )

best_cut_ = float(threshs[np.argwhere(np.diff(np.sign(rec - prec))).flatten()])

'''plt.plot( threshs ,rec , color = 'orange' , label = 'recall' )
plt.plot( threshs ,prec ,  color = 'blue' , label = 'precision' )
plt.plot( [ best_cut_,best_cut_ ] , [-0.05,(prec).max()*2] ,"-.r",label='Best Cut : {:2.5f}'.format(best_cut_) )
#plt.ylim(-0.03,1.05)
plt.xlabel(  'THRESHOLDS' )
plt.legend(loc="best")
plt.tight_layout()
plt.show()
plt.close()
'''
fig= plt.figure( figsize=(10,10) )
plt.plot( threshs , prec*rec ,  color = 'blue' )
plt.plot( [ best_cut,best_cut ] , [-0.4,(prec * rec).max()*2] ,"-.r",label='Best Cut : {:2.5f}'.format(best_cut) )
plt.plot( [-0.1,1.1] , [ (prec*rec).max(),(prec*rec).max()], "-g",label='precision*recall : {:2.5f}'.format((prec*rec).max()) )
plt.ylabel( 'PRECISION * RECALL' )
plt.xlabel(    'THRESHOLDS'    )
#plt.ylim( -0.002 , 0.105 )
#plt.xlim( -0.01 , 1.05)
plt.legend(loc="best", fontsize=20)
#plt.style.use(hep.style.CMS)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.show()
#plt.savefig('/home/matheus/test/prec_rec_normalized.png')
plt.close()

fig= plt.figure( figsize=(10,10) )
plt.plot( threshs , f1 ,  color = 'green' )
plt.plot( [ best_cut,best_cut ] , [-0.4,(f1).max()*2] ,"-.r",label='Best Cut : {:2.5f}'.format(best_cut) )
plt.plot( [-0.1,1.1] , [ (f1).max(),(f1).max()], "-b",label='F1_SCORE : {:2.5f}'.format((f1).max()) )
plt.ylabel(  'F1_SCORE'  )
plt.xlabel( 'THRESHOLDS' )
#plt.ylim( -0.002 , 0.35 )
#plt.xlim( -0.01 , 1.05)
plt.legend(loc="best",fontsize=20)
#plt.style.use(hep.style.CMS)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.show()
#plt.savefig('/home/matheus/test/f1_normalized.png')
plt.close()

In [ ]:
fig= plt.figure( figsize=(10,10) )

confu_matrix = confusion_matrix( y_test_ , y_pred_cut , sample_weight = dataset_test.Weight*Norm_Test )
confu_matrix_DY = ((DataFrame(confu_matrix[0,:]).T).rename(columns = {0: 'Background', 1: 'Signal'}).T).rename(columns = {0: 'Background'}).T
confu_matrix_sig = ((DataFrame(confu_matrix[1,:]).T).rename(columns = {0: 'Background', 1: 'Signal'}).T).rename(columns = {0:'Signal'}).T
cunfusion_matrix = pd.concat([confu_matrix_DY,confu_matrix_sig],axis=0)
sns.heatmap(cunfusion_matrix.astype('int'), annot=True, fmt="d", cmap = 'summer')
plt.title('Confusion Matrix - cut: {:2.5f}'.format(best_cut) )
plt.xlabel('Predicted')
plt.ylabel('Expected')
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/matrix_normalized.png')
plt.close() 

In [ ]:
fig= plt.figure( figsize=(15,15) )

counts_1, bin_edges_1 = np.histogram( predict_proba_LGBM_Data, bins=np.linspace(0,1,50))
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
plt.errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

hist_Exclu = plt.hist(predict_proba_LGBM_exclusivo, bins=np.linspace(0,1,50), histtype='step', color = 'green', label='Exclusivo', weights = MonteCarlo_Exclusive[ 'Weight' ]*Norm_Test)
hist_SemiExclu = plt.hist(predict_proba_LGBM_semiexclusivo, bins=np.linspace(0,1,50), histtype='step',color = 'orange',label='Semi-Exclusivo',  weights = MonteCarlo_SemiExclusive[ 'Weight' ]*Norm_Test )
hist_DY = plt.hist(predict_proba_LGBM_background, bins=np.linspace(0,1,50), histtype='step',label='Data-driven Background', color = 'blue', weights = MonteCarlo_DrellYan[ 'Weight' ]*Norm_Test )
#hist_Data = plt.hist(predict_proba_LGBM_Data, bins=np.linspace(0,1,50), histtype='step',label='Data-2017', color = 'violet' )
plt.plot([best_cut,best_cut],[0,10e10],"-.r",label='Best Cut : {:2.5f}'.format(best_cut))
plt.yscale('log')
plt.ylim(10e-4,10e8)
plt.xlabel("Probability")
plt.ylabel("Events")
plt.legend(loc="upper left", fontsize=20)
plt.text(best_cut+0.02,10e2, 'SIGNAL REGION', color = 'red', rotation = 90, fontsize = 30)
plt.text(0.5,15e6, "Purity: {:2.2f}%".format(100*precision_score(y_test_,y_pred_cut, sample_weight = dataset_test.Weight*Norm_Test)),fontsize = 20)
plt.text(0.5,20e5, "Efficiency: {:2.2f}%".format(100*recall_score(y_test_,y_pred_cut, sample_weight = dataset_test.Weight*Norm_Test)),fontsize = 20)
plt.text(0.5,30e4, "F1_Score: {:2.2f}%".format(100*f1_score(y_test_,y_pred_cut, sample_weight = dataset_test.Weight*Norm_Test)),fontsize = 20)
plt.text(0.5,40e3, "ROC AUC: {:2.2f}%".format(100*roc_auc_score(y_test_,y_pred_cut, sample_weight = dataset_test.Weight*Norm_Test)),fontsize = 20)
plt.text(0.5,60e2, "Acuracy: {:2.2f}%".format(100*accuracy_score(y_test_,y_pred_cut, sample_weight = dataset_test.Weight*Norm_Test)),fontsize = 20)
#hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$')
#plt.style.use(hep.style.CMS)
plt.show()
plt.tight_layout()
#plt.savefig('/home/matheus/test/discrimante_normalized.png')
plt.close()


In [ ]:
class_pred_DY = predict_proba_LGBM_background >= best_cut
class_pred_Exclusive = predict_proba_LGBM_exclusivo >= best_cut
class_pred_SemiExclusive = predict_proba_LGBM_semiexclusivo >= best_cut
class_pred_Data = predict_proba_LGBM_Data >= best_cut

MonteCarlo_Exclusive['target'] = DataFrame((map(int, class_pred_Exclusive)))
MonteCarlo_SemiExclusive['target'] = DataFrame((map(int, class_pred_SemiExclusive)))
MonteCarlo_DrellYan['target'] = DataFrame((map(int, class_pred_DY)))
df_data['target'] = DataFrame((map(int, class_pred_Data)))

events_select_DY = MonteCarlo_DrellYan.loc[MonteCarlo_DrellYan['target'] == 1]
events_select_exclusive = MonteCarlo_Exclusive.loc[MonteCarlo_Exclusive['target'] == 1]
events_select_semiexclu = MonteCarlo_SemiExclusive.loc[MonteCarlo_SemiExclusive['target'] == 1]
events_select_Data = df_data.loc[df_data['target'] == 1]

print('# eventos Data-Drive Background', (events_select_DY['Weight']*Norm_Test).sum())
print('# eventos Semi-Exclusive', (events_select_semiexclu['Weight']*Norm_Test).sum())
print('# eventos Exclusivo', (events_select_exclusive['Weight']*Norm_Test).sum())
print('# eventos Data', len(events_select_Data))
events_select = pd.concat([events_select_DY,events_select_exclusive,events_select_semiexclu],axis=0)

In [ ]:
with h5py.File( '/content/gdrive/My Drive/pps_ML/data_after_train' + '.h5', 'w') as f:
    dset = f.create_dataset( 'data_after', data=events_select_Data )
with h5py.File( '/content/gdrive/My Drive/pps_ML/exclus_after_train' + '.h5', 'w') as f:
    dset = f.create_dataset( 'exclu_after', data=events_select_exclusive )
with h5py.File( '/content/gdrive/My Drive/pps_ML/semiexlu_after_train' + '.h5', 'w') as f:
    dset = f.create_dataset( 'semiexc_after', data=events_select_semiexclu )
with h5py.File( '/content/gdrive/My Drive/pps_ML/backgr_after_train' + '.h5', 'w') as f:
    dset = f.create_dataset( 'back_after', data=events_select_DY )            


In [ ]:
fig= plt.figure( figsize=(10,10) )

msk_bkg = ( y_pred_cut == 0 )
msk_sig = ( y_pred_cut == 1 )

plt.plot( dataset_test[ "Xi" ][ msk_bkg ], dataset_test[ "XiMuMu" ][ msk_bkg ], 'ro', label = 'Background Previsto' )
plt.plot( dataset_test[ "Xi" ][ msk_sig ], dataset_test[ "XiMuMu" ][ msk_sig ], 'bo', label = 'Signal Previsto' )
plt.plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
plt.plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
plt.plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
plt.xlim(0.,0.15)
plt.ylim(0.,0.15)
plt.legend(loc='best')
plt.xlabel(r'$\xi_{proton_{RP}}$', fontsize=17)
plt.ylabel(r'$\xi_{\mu^{+}\mu^{-}}$', fontsize=17)

In [ ]:
y_sig_pred = np.concatenate([class_pred_Exclusive,class_pred_SemiExclusive])
y_bkg_pred = class_pred_DY

In [ ]:
MonteCarlo_Signal = pd.concat([MonteCarlo_Exclusive,MonteCarlo_SemiExclusive])

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

msk_bkg_1 = None
msk_bkg_2 = None
msk_sig_1 = None
msk_sig_2 = None
if proton_selection == "SingleRP":
    msk_bkg_1 = ( MonteCarlo_DrellYan[ "RPId1" ] == 23 ) & ( y_bkg_pred == 0 )
    msk_bkg_2 = ( MonteCarlo_DrellYan[ "RPId1" ] == 123 ) & ( y_bkg_pred == 0 )
    msk_sig_1 = ( MonteCarlo_Signal[ "RPId1" ] == 23 ) & ( y_sig_pred == 1 )
    msk_sig_2 = ( MonteCarlo_Signal[ "RPId1" ] == 123 ) & ( y_sig_pred == 1 )
elif proton_selection == "MultiRP":
    msk_bkg_1 = ( MonteCarlo_DrellYan[ "Arm" ] == 0 ) & ( y_bkg_pred == 0 )
    msk_bkg_2 = ( MonteCarlo_DrellYan[ "Arm" ] == 1 ) & ( y_bkg_pred == 0 )
    msk_sig_1 = ( MonteCarlo_Signal[ "Arm" ] == 0 ) & ( y_sig_pred == 1 )
    msk_sig_2 = ( MonteCarlo_Signal[ "Arm" ] == 1 ) & ( y_sig_pred == 1 )
    
axes[0].plot( MonteCarlo_DrellYan[ "Xi" ][ msk_bkg_1 ], MonteCarlo_DrellYan[ "XiMuMuPlus" ][ msk_bkg_1 ], 'ro', label = 'Brackground Previsto' )
axes[0].plot( MonteCarlo_Signal[ "Xi" ][ msk_sig_1 ], MonteCarlo_Signal[ "XiMuMuPlus" ][ msk_sig_1 ], 'bo', label = "Signal Previsto" )
axes[0].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[0].set_xlim(0.,0.15)
axes[0].set_ylim(0.,0.15)
axes[0].set_xlabel(r'$\xi_{proton_{RP}}$', fontsize=20)
axes[0].set_ylabel(r'$\xi_{\mu^{+}\mu^{-}}$', fontsize=20)
axes[0].set_title('Setor 45 - MultiRP')
axes[0].legend(loc='best', fontsize=20)

axes[1].plot( MonteCarlo_DrellYan[ "Xi" ][ msk_bkg_2 ], MonteCarlo_DrellYan[ "XiMuMuMinus" ][ msk_bkg_2 ], 'ro',label = 'Brackground Previsto' )
axes[1].plot( MonteCarlo_Signal[ "Xi" ][ msk_sig_2 ], MonteCarlo_Signal[ "XiMuMuMinus" ][ msk_sig_2 ], 'bo', label = "Signal Previsto" )
axes[1].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[1].set_xlim(0.,0.15)
axes[1].set_ylim(0.,0.15)
axes[1].set_xlabel(r'$\xi_{proton_{RP}}$', fontsize=20)
axes[1].set_ylabel(r'$\xi_{\mu^{+}\mu^{-}}$', fontsize=20)
axes[1].set_title('Setor 56 - MultiRP')
axes[1].legend(loc='best',fontsize=20)

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

msk_bkg_1 = None
msk_bkg_2 = None
msk_bkgerr_1 = None
msk_bkgerr_2 = None
if proton_selection == "SingleRP":
    msk_bkg_1 = ( MonteCarlo_DrellYan[ "RPId1" ] == 23 ) & ( y_bkg_pred == 0 )
    msk_bkg_2 = ( MonteCarlo_DrellYan[ "RPId1" ] == 123 ) & ( y_bkg_pred == 0 )
    msk_bkgerr_1 = ( MonteCarlo_DrellYan[ "RPId1" ] == 23 ) & ( y_bkg_pred == 1 )
    msk_bkgerr_2 = ( MonteCarlo_DrellYan[ "RPId1" ] == 123 ) & ( y_bkg_pred == 1 )
elif proton_selection == "MultiRP":
    msk_bkg_1 = ( MonteCarlo_DrellYan[ "Arm" ] == 0 ) & ( y_bkg_pred == 0 )
    msk_bkg_2 = ( MonteCarlo_DrellYan[ "Arm" ] == 1 ) & ( y_bkg_pred == 0 )
    msk_bkgerr_1 = ( MonteCarlo_DrellYan[ "Arm" ] == 0 ) & ( y_bkg_pred == 1 )
    msk_bkgerr_2 = ( MonteCarlo_DrellYan[ "Arm" ] == 1 ) & ( y_bkg_pred == 1 )
    
axes[0].plot( MonteCarlo_DrellYan[ "Xi" ][ msk_bkg_1 ], MonteCarlo_DrellYan[ "XiMuMuPlus" ][ msk_bkg_1 ], 'ro' )
axes[0].plot( MonteCarlo_DrellYan[ "Xi" ][ msk_bkgerr_1 ], MonteCarlo_DrellYan[ "XiMuMuPlus" ][ msk_bkgerr_1 ], 'o', color='orange' )
axes[0].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[0].set_xlim(0.,0.15)
axes[0].set_ylim(0.,0.15)

axes[1].plot( MonteCarlo_DrellYan[ "Xi" ][ msk_bkg_2 ], MonteCarlo_DrellYan[ "XiMuMuMinus" ][ msk_bkg_2 ], 'ro' )
axes[1].plot( MonteCarlo_DrellYan[ "Xi" ][ msk_bkgerr_2 ], MonteCarlo_DrellYan[ "XiMuMuMinus" ][ msk_bkgerr_2 ], 'o', color='orange' )
axes[1].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[1].set_xlim(0.,0.15)
axes[1].set_ylim(0.,0.15)

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

msk_sigerr_1 = None
msk_sigerr_2 = None
msk_sig_1 = None
msk_sig_2 = None
if proton_selection == "SingleRP":
    msk_sigerr_1 = ( MonteCarlo_Signal[ "RPId1" ] == 23 ) & ( y_sig_pred == 0 )
    msk_sigerr_2 = ( MonteCarlo_Signal[ "RPId1" ] == 123 ) & ( y_sig_pred == 0 )
    msk_sig_1 = ( MonteCarlo_Signal[ "RPId1" ] == 23 ) & ( y_sig_pred == 1 )
    msk_sig_2 = ( MonteCarlo_Signal[ "RPId1" ] == 123 ) & ( y_sig_pred == 1 )
elif proton_selection == "MultiRP":
    msk_sigerr_1 = ( MonteCarlo_Signal[ "Arm" ] == 0 ) & ( y_sig_pred == 0 )
    msk_sigerr_2 = ( MonteCarlo_Signal[ "Arm" ] == 1 ) & ( y_sig_pred == 0 )
    msk_sig_1 = ( MonteCarlo_Signal[ "Arm" ] == 0 ) & ( y_sig_pred == 1 )
    msk_sig_2 = ( MonteCarlo_Signal[ "Arm" ] == 1 ) & ( y_sig_pred == 1 )

axes[0].plot( MonteCarlo_Signal[ "Xi" ][ msk_sigerr_1 ], MonteCarlo_Signal[ "XiMuMuPlus" ][ msk_sigerr_1 ], 'o', color='orange' )
axes[0].plot( MonteCarlo_Signal[ "Xi" ][ msk_sig_1 ], MonteCarlo_Signal[ "XiMuMuPlus" ][ msk_sig_1 ], 'bo' )
axes[0].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[0].set_xlim(0.,0.15)
axes[0].set_ylim(0.,0.15)
axes[1].plot( MonteCarlo_Signal[ "Xi" ][ msk_sigerr_2 ], MonteCarlo_Signal[ "XiMuMuMinus" ][ msk_sigerr_2 ], 'o', color='orange' )
axes[1].plot( MonteCarlo_Signal[ "Xi" ][ msk_sig_2 ], MonteCarlo_Signal[ "XiMuMuMinus" ][ msk_sig_2 ], 'bo' )
axes[1].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[1].set_xlim(0.,0.15)
axes[1].set_ylim(0.,0.15)

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( dataset_test[ "Acopl" ][ y_pred_cut == 1 ], histtype='step', color='skyblue', bins=50, range=(0.,0.02), density=True, label="Signal" )
plt.hist( dataset_test[ "Acopl" ][ y_pred_cut == 0 ], histtype='step', color='orange', bins=50, range=(0.,0.02), density=True, label="Background" )
plt.xlabel('Acoplanaridade')
plt.legend(loc="best")

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( dataset_test[ "ExtraPfCands" ][ y_pred_cut == 1 ], histtype='step', color='skyblue', bins=50, range=(0,50), density=True, label="Signal" )
plt.hist( dataset_test[ "ExtraPfCands" ][ y_pred_cut == 0 ], histtype='step', color='orange', bins=50, range=(0,50), density=True,label="Background" )
plt.xlabel('Traços Extras')
plt.legend(loc="best")

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( dataset_test[ "pt" ][ y_pred_cut == 1 ], histtype='step', color='skyblue', bins=50, range=(0,50), density=True, label='Signal' )
plt.hist( dataset_test[ "pt" ][ y_pred_cut == 0 ], histtype='step', color='orange', bins=50, range=(0,50), density=True, label="Background" )
plt.xlabel("$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$")
plt.legend(loc="best")

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( dataset_test[ "InvMass" ][ y_pred_cut == 1 ], histtype='step', color='skyblue', bins=50, range=(0,600), density=True, label='Signal' )
plt.hist( dataset_test[ "InvMass" ][ y_pred_cut == 0 ], histtype='step', color='orange', bins=50, range=(0,600), density=True,label="Background" )
plt.xlabel("$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$")
plt.legend(loc="best")

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

msk_bkg_1 = None
msk_bkg_2 = None
msk_sig_1 = None
msk_sig_2 = None
if proton_selection == "SingleRP":
    msk_bkg_1 = ( df_data[ "RPId1" ] == 23 ) & ( y_data == 0 )
    msk_bkg_2 = ( df_data[ "RPId1" ] == 123 ) & ( y_data == 0 )
    msk_sig_1 = ( df_data[ "RPId1" ] == 23 ) & ( y_data == 1 )
    msk_sig_2 = ( df_data[ "RPId1" ] == 123 ) & ( y_data == 1 )
elif proton_selection == "MultiRP":
    msk_bkg_1 = ( df_data[ "Arm" ] == 0 ) & ( class_pred_Data == 0 )
    msk_bkg_2 = ( df_data[ "Arm" ] == 1 ) & ( class_pred_Data == 0 )
    msk_sig_1 = ( df_data[ "Arm" ] == 0 ) & ( class_pred_Data == 1 )
    msk_sig_2 = ( df_data[ "Arm" ] == 1 ) & ( class_pred_Data == 1 )

axes[0].plot( df_data[ "Xi" ][ msk_bkg_1 ], df_data[ "XiMuMuPlus" ][ msk_bkg_1 ], 'ro', label="Eventos não Correlatos" )
axes[0].plot( df_data[ "Xi" ][ msk_sig_1 ], df_data[ "XiMuMuPlus" ][ msk_sig_1 ], 'bo',label="Eventos Correlatos" )
axes[0].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[0].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[0].set_xlim(0.,0.15)
axes[0].set_ylim(0.,0.15)
axes[0].set_xlabel(r'$\xi_{proton_{RP}}$',  fontsize=20)
axes[0].set_ylabel(r'$\xi_{\mu^{+}\mu_{-}}$',  fontsize=20)
axes[0].axvspan(0, 0.02, color='y', alpha=0.5, lw=0, label="Sem aceitação para qualquer RP")
axes[0].axvspan(0.02, 0.03, color='g', alpha=0.5, lw=0, label="Sem aceitação para RP correto")
axes[0].legend(loc="lower left", fontsize=17)
axes[0].set_title('Setor 45 - MultiRP',  fontsize=20)

axes[1].plot( df_data[ "Xi" ][ msk_bkg_2 ], df_data[ "XiMuMuMinus" ][ msk_bkg_2 ], 'ro', label="Eventos não Correlatos" )
axes[1].plot( df_data[ "Xi" ][ msk_sig_2 ], df_data[ "XiMuMuMinus" ][ msk_sig_2 ], 'bo', label="Eventos Correlatos" )
axes[1].plot( (0.,0.15), (0.,0.15), 'k--', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,0.90*0.15), 'k:', linewidth=1 )
axes[1].plot( (0.,0.15), (0.,1.10*0.15), 'k:', linewidth=1 )
axes[1].set_xlim(0.,0.15)
axes[1].set_ylim(0.,0.15)
axes[1].set_xlabel(r'$\xi_{proton_{RP}}$',  fontsize=20)
axes[1].set_ylabel(r'$\xi_{\mu^{+}\mu^{-}}$',  fontsize=20)
axes[1].axvspan(0, 0.02, color='y', alpha=0.5, lw=0, label="Sem aceitação para qualquer RP")
axes[1].axvspan(0.02, 0.03, color='g', alpha=0.5, lw=0,  label="Sem aceitação para RP correto")
axes[1].legend(loc="lower left", fontsize=17)
axes[1].set_title('Setor 56 - MultiRP', fontsize=20)


In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( df_data[ "Acopl" ][ class_pred_Data == 1 ], histtype='step', color='skyblue', bins=50, range=(0.,0.02), density=True )
plt.hist( df_data[ "Acopl" ][ class_pred_Data == 0 ], histtype='step', color='orange', bins=50, range=(0.,0.02), density=True )

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( df_data[ "ExtraPfCands" ][ class_pred_Data == 1 ], histtype='step', color='skyblue', bins=50, range=(0,50), density=True )
plt.hist( df_data[ "ExtraPfCands" ][ class_pred_Data == 0 ], histtype='step', color='orange', bins=50, range=(0,50), density=True )

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( df_data[ "InvMass" ][ class_pred_Data == 1 ], histtype='step', color='skyblue', bins=50, range=(0,400), density=True )
plt.hist( df_data[ "InvMass" ][ class_pred_Data == 0 ], histtype='step', color='orange', bins=50, range=(0,400), density=True )

In [ ]:
fig = plt.figure( figsize=(10,10) )
plt.hist( df_data[ "pt" ][ class_pred_Data == 1 ], histtype='step', color='skyblue', bins=50, range=(0,50), density=True )
plt.hist( df_data[ "pt" ][ class_pred_Data == 0 ], histtype='step', color='orange', bins=50, range=(0,50), density=True )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

#var_ = "Xi"
#bins_ = 10
#range_ = (0.,0.2)

#var_ = "ThX"
#bins_ = 10
#range_ = (-0.0005,0.0005)

#var_ = "ThY"
#bins_ = 10
#range_ = (-0.0005,0.0005)

#var_ = "T"
#bins_ = 10
#range_ = (-4.,0.)

#var_ = "Time"
#bins_ = 20
#range_ = (-0.5,0.5)

#var_ = "pt"
#bins_ = 10
#range_ = (0.,10.)

var_ = "InvMass"
bins_ = 10
range_ = (100.,500.)

#var_ = "Acopl"
#bins_ = 10
#range_ = (0.,0.01)

#var_ = "ExtraPfCands"
#bins_ = 20
#range_ = (0,20)

msk_data_1 = None
msk_data_2 = None
if proton_selection == "SingleRP":
    msk_data_1 = ( df_data[ "RPId1" ] == 23 ) & ( y_data == 1 )
    msk_data_2 = ( df_data[ "RPId1" ] == 123 ) & ( y_data == 1 )
elif proton_selection == "MultiRP":
    msk_data_1 = ( df_data[ "Arm" ] == 0 ) & ( class_pred_Data == 1 )
    msk_data_2 = ( df_data[ "Arm" ] == 1 ) & ( class_pred_Data == 1 )

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_)
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

msk_bkg_pred_1 = None
msk_bkg_pred_2 = None
if proton_selection == "SingleRP":
    msk_bkg_pred_1 = ( MonteCarlo_DrellYan[ "RPId1" ] == 23 ) & ( y_bkg_pred == 1 )
    msk_sgex_pred_1 = ( MonteCarlo_Exclusive[ "RPId1" ] == 23 ) & ( class_pred_Exclusive == 1 )
    msk_sgsex_pred_1 = ( MonteCarlo_SemiExclusive[ "RPId1" ] == 23 ) & ( class_pred_SemiExclusive == 1 )

    msk_bkg_pred_2 = ( MonteCarlo_DrellYan[ "RPId1" ] == 123 ) & ( y_bkg_pred == 1 )
    msk_sgex_pred_2 = ( MonteCarlo_Exclusive[ "RPId1" ] == 123 ) & ( class_pred_Exclusive == 1 )
    msk_sgsex_pred_2 = ( MonteCarlo_SemiExclusive[ "RPId1" ] == 123 ) & ( class_pred_SemiExclusive == 1 )
elif proton_selection == "MultiRP":
    msk_bkg_pred_1 = ( MonteCarlo_DrellYan[ "Arm" ] == 0 ) & ( y_bkg_pred == 1 )
    msk_sgex_pred_1 = ( MonteCarlo_Exclusive[ "Arm" ] == 0 ) & ( class_pred_Exclusive == 1 )
    msk_sgsex_pred_1 = ( MonteCarlo_SemiExclusive[ "Arm" ] == 0 ) & ( class_pred_SemiExclusive == 1 )

    msk_bkg_pred_2 = ( MonteCarlo_DrellYan[ "Arm" ] == 1 ) & ( y_bkg_pred == 1 )    
    msk_sgex_pred_2 = ( MonteCarlo_Exclusive[ "Arm" ] == 1 ) & ( class_pred_Exclusive == 1 )
    msk_sgsex_pred_2 = ( MonteCarlo_SemiExclusive[ "Arm" ] == 1 ) & ( class_pred_SemiExclusive == 1 )

weights_1_back = None
if resample_factor > 1:
    weights_1_back = np.full_like( MonteCarlo_DrellYan[ "Xi" ][ msk_bkg_pred_1 ], ( 1./resample_factor ) )
weights_1_sgexclu = None
if resample_factor > 1:
    weights_1_sgexclu = np.full_like( MonteCarlo_Exclusive[ "Xi" ][ msk_sgex_pred_1 ], ( 1./resample_factor ) )
weights_1_sgseexlu = None
if resample_factor > 1:
    weights_1_sgseexlu = np.full_like( MonteCarlo_SemiExclusive[ "Xi" ][ msk_sgsex_pred_1 ], ( 1./resample_factor ) )

weights_2_back = None
if resample_factor > 1:
    weights_2_back = np.full_like( MonteCarlo_DrellYan[ "Xi" ][ msk_bkg_pred_2 ], ( 1./resample_factor ) )
weights_2_sgexlu = None
if resample_factor > 1:
    weights_2 = np.full_like( MonteCarlo_Exclusive[ "Xi" ][ msk_sgex_pred_2 ], ( 1./resample_factor ) )
weights_2_sgsemi = None
if resample_factor > 1:
    weights_2 = np.full_like( MonteCarlo_SemiExclusive[ "Xi" ][ msk_sgsex_pred_2 ], ( 1./resample_factor ) )

axes[0].hist( MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_1 ], stacked = True, bins=bins_, range=range_, weights = [weights_1_back*Norm_Test], label = ["Data-driven Background"] )
axes[0].legend()
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$")
axes[0].set_ylabel('Eventos')

axes[1].hist( MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_2 ], stacked = True, bins=bins_, range=range_, weights = [weights_2_back*Norm_Test], label = ["Data-driven Background"] )
axes[1].legend()
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$")
axes[1].set_ylabel('Eventos')


In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

var_ = "InvMass"
bins_ = 10
range_ = (0.,500.)

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_ )
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

axes[0].hist( [ MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_1 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_1 ]], stacked = True, bins=bins_, range=range_, weights =  [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_1 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_1 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[0].legend(loc='best',fontsize=18)
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$")
axes[0].set_ylabel('Eventos')

axes[1].hist( [MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_2 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_2 ]], stacked = True, bins=bins_, range=range_, weights = [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_2 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_2 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[1].legend(loc='best',fontsize=18)
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$")
axes[1].set_ylabel('Eventos')



In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

var_ = "pt"
bins_ = 10
range_ = (0.,10.)

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_ )
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

axes[0].hist( MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_1 ], stacked = True, bins=bins_, range=range_, weights = [weights_1_back*Norm_Test], label = ["Data-driven Background"] )
axes[0].legend(loc='best',fontsize=18)
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$")
axes[0].set_ylabel('Eventos')

axes[1].hist( MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_2 ], stacked = True, bins=bins_, range=range_, weights = [weights_2_back*Norm_Test], label = ["Data-driven Background"] )
axes[1].legend(loc='best',fontsize=18)
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$")
axes[1].set_ylabel('Eventos')

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

var_ = "pt"
bins_ = 10
range_ = (0.,15.)

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_ )
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

axes[0].hist( [ MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_1 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_1 ]], stacked = True, bins=bins_, range=range_, weights =  [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_1 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_1 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[0].legend(loc='best',fontsize=18)
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$")
axes[0].set_ylabel('Eventos')

axes[1].hist( [MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_2 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_2 ]], stacked = True, bins=bins_, range=range_, weights = [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_2 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_2 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[1].legend(loc='best',fontsize=18)
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$")
axes[1].set_ylabel('Eventos')

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

var_ = "ExtraPfCands"
bins_ = 20
range_ = (0,20)

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_ )
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

axes[0].hist(MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_1 ], stacked = True, bins=bins_, range=range_, weights = [weights_1_back*Norm_Test], label = ["Data-driven Background"] )
axes[0].legend(loc='best',fontsize=18)
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("Traços Extras")
axes[0].set_ylabel('Eventos')

axes[1].hist( [MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_2 ]], stacked = True, bins=bins_, range=range_, weights = [weights_2_back*Norm_Test], label = ["Data-driven Background"] )
axes[1].legend(loc='best',fontsize=18)
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("Traços Extras")
axes[1].set_ylabel('Eventos')

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

var_ = "ExtraPfCands"
bins_ = 10
range_ = (0.,15.)

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_ )
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

axes[0].hist( [ MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_1 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_1 ]], stacked = True, bins=bins_, range=range_, weights =  [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_1 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_1 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[0].legend(loc='best',fontsize=18)
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("Traços Extras")
axes[0].set_ylabel('Eventos')

axes[1].hist( [MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_2 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_2 ]], stacked = True, bins=bins_, range=range_, weights = [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_2 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_2 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[1].legend(loc='best',fontsize=18)
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("Traços Extras")
axes[1].set_ylabel('Eventos')

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

var_ = "Acopl"
bins_ = 10
range_ = (0.,0.01)

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_ )
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

axes[0].hist( MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_1 ], stacked = True, bins=bins_, range=range_, weights = [weights_1_back*Norm_Test], label = ["Data-driven Background"] )
axes[0].legend(loc='best',fontsize=18)
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("Acoplanaridade")
axes[0].set_ylabel('Eventos')

axes[1].hist( MonteCarlo_DrellYan[ var_ ][ msk_bkg_pred_2 ], stacked = True, bins=bins_, range=range_, weights = [weights_2_back*Norm_Test], label = ["Data-driven Background"] )
axes[1].legend(loc='best',fontsize=18)
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("Acoplanaridade")
axes[1].set_ylabel('Eventos')

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(20,10) )

var_ = "Acopl"
bins_ = 10
range_ = (0.,0.009)

counts_1, bin_edges_1 = np.histogram( df_data[ var_ ][ msk_data_1 ], bins=bins_, range=range_ )
errors_1 = np.sqrt( counts_1 )
bin_centres_1 = ( bin_edges_1[:-1] + bin_edges_1[1:] ) / 2.
axes[0].errorbar(bin_centres_1, counts_1, yerr=errors_1, fmt='o', label = 'Data-2017', color = 'black')

counts_2, bin_edges_2 = np.histogram( df_data[ var_ ][ msk_data_2 ], bins=bins_, range=range_ )
errors_2 = np.sqrt( counts_2 )
bin_centres_2 = ( bin_edges_2[:-1] + bin_edges_2[1:] ) / 2.
axes[1].errorbar(bin_centres_2, counts_2, yerr=errors_2, fmt='o', label = 'Data-2017', color = 'black')

idx_ymax_ = np.argmax( np.concatenate( [counts_1, counts_2] ) )
y_max = np.concatenate( [counts_1, counts_2] )[idx_ymax_] + 2*np.concatenate( [errors_1, errors_2] )[idx_ymax_]
print ( "y max. = {}".format(y_max) )

axes[0].hist( [ MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_1 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_1 ]], stacked = True, bins=bins_, range=range_, weights =  [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_1 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_1 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[0].legend(loc='best',fontsize=18)
axes[0].set_title("MultiRP Sector45", fontsize=18)
axes[0].set_xlabel("Acoplanaridade")
axes[0].set_ylabel('Eventos')

axes[1].hist( [MonteCarlo_Exclusive[ var_ ][ msk_sgex_pred_2 ],
               MonteCarlo_SemiExclusive[ var_ ][ msk_sgsex_pred_2 ]], stacked = True, bins=bins_, range=range_, weights = [MonteCarlo_Exclusive.Weight[ msk_sgex_pred_2 ]*Norm_Test,
                                                                                                                           MonteCarlo_SemiExclusive.Weight[ msk_sgsex_pred_2 ]*Norm_Test], label = ["Exclusivo","Semi-Exclusivo"] )
axes[1].legend(loc='best',fontsize=18)
axes[1].set_title("MultiRP Sector56", fontsize=18)
axes[1].set_xlabel("Acoplanaridade")
axes[1].set_ylabel('Eventos')

In [ ]:
!python3 -m pip install --user coffea numba lz4 uproot six awkward1 mplhep cloudpickle packaging uproot-methods pandas numpy uproot4 scipy awkward tqdm

In [ ]:
import coffea.hist as hist

# HISTOGRAMAS PARA O SETOR 45

In [ ]:
histo = hist.Hist("Events",
        hist.Cat("sample", ""),
        hist.Bin("pt", "$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$", 15, 0, 10),
        hist.Bin("massa_inv", "$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$", 15, 0, 500),
        hist.Bin("acopla", "Acoplanaridade", 20, 0, 0.004),   
        hist.Bin("extratracks", "Traços Extras", 20, 0, 40)
        )
histo.fill(
    sample = "Data-driven Background",
    pt = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_1 ].pt),
    massa_inv = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_1 ].InvMass),
    acopla = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_1 ].Acopl),
    extratracks = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_1 ].ExtraPfCands),
    weight = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_1 ].Weight*Norm_Test)
    )
histo.fill(
    sample = "Exclusivo",
    pt = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_1 ].pt),
    massa_inv = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_1 ].InvMass),
    acopla = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_1 ].Acopl),   
    extratracks = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_1 ].ExtraPfCands), 
    weight = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_1 ].Weight*Norm_Test)
    )
histo.fill(
    sample = "Semi-Exclusivo",
    pt = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_1 ].pt),
    massa_inv = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_1 ].InvMass),
    acopla = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_1 ].Acopl),   
    extratracks = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_1 ].ExtraPfCands), 
    weight = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_1 ].Weight*Norm_Test)
    )
histo.fill(
    sample = "Data-2017",
    pt = np.array(df_data[msk_data_1].pt),
    massa_inv = np.array(df_data[msk_data_1].InvMass),
    acopla = np.array(df_data[msk_data_1].Acopl),  
    extratracks = np.array(df_data[ msk_data_1 ].ExtraPfCands) 
    )

plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
})

fill_opts = {
    'edgecolor': (0,0,0,0.3),
    'alpha': 0.8
}
error_opts = {
    'label': 'Stat. Unc.',
    'hatch': '///',
    'facecolor': 'none',
    'edgecolor': (0,0,0,1),
    'linewidth': 0
}
data_err_opts = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1,
}

import re
notdata = re.compile('(?!Data-2017)')

In [ ]:
fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True
)
fig.subplots_adjust(hspace=.07)

pt1hist = histo.sum("extratracks", "massa_inv", 'acopla')

hist.plot1d(
    pt1hist['Data-2017'],
    overlay="sample",
    clear=True,
    ax=ax,
    error_opts=data_err_opts
    )
hist.plot1d(
    pt1hist[notdata],
    overlay='sample',
    stack=True,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],
    clear=False,
    ax=ax,
    line_opts=False,
    fill_opts=fill_opts,
    error_opts=error_opts
    )

#plt.yscale('log')
#plt.style.use([hep.style.ROOT, hep.style.firamath])
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 6)
ax.set_xlabel(None)
leg = ax.legend()

hist.plotratio(
    num=pt1hist['Data-2017'].sum("sample"),
    denom=pt1hist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_pt_normalized.png')
plt.close()

In [ ]:
masshist = histo.sum("pt", "extratracks", 'acopla')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    masshist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    masshist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
#plt.yscale('log')
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 8)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=masshist['Data-2017'].sum("sample"),
    denom=masshist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

In [ ]:
extrahist = histo.sum("pt", "massa_inv", 'acopla')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    extrahist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    extrahist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.ylim(0, 1750)
#plt.yscale('log')
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 23)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=extrahist['Data-2017'].sum("sample"),
    denom=extrahist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

In [ ]:
acoplahist = histo.sum("pt", "massa_inv", 'extratracks')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    acoplahist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    acoplahist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 5)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=acoplahist['Data-2017'].sum("sample"),
    denom=acoplahist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

# HISTOGRAMAS PARA O SETOR 56

In [ ]:
histo = hist.Hist("Events",
        hist.Cat("sample", ""),
        hist.Bin("pt", "$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$", 15, 0, 10),
        hist.Bin("massa_inv", "$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$", 15, 0, 500),
        hist.Bin("acopla", "Acoplanaridade", 20, 0, 0.004),   
        hist.Bin("extratracks", "Traços Extras", 20, 0, 40)
        )
histo.fill(
    sample = "Data-driven Background",
    pt = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_2 ].pt),
    massa_inv = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_2 ].InvMass),
    acopla = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_2 ].Acopl),
    extratracks = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_2 ].ExtraPfCands),
    weight = np.array(MonteCarlo_DrellYan[ msk_bkg_pred_2 ].Weight*Norm_Test)
    )
histo.fill(
    sample = "Exclusivo",
    pt = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_2 ].pt),
    massa_inv = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_2 ].InvMass),
    acopla = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_2 ].Acopl),   
    extratracks = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_2 ].ExtraPfCands), 
    weight = np.array(MonteCarlo_Exclusive[ msk_sgex_pred_2 ].Weight*Norm_Test)
    )
histo.fill(
    sample = "Semi-Exclusivo",
    pt = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_2 ].pt),
    massa_inv = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_2 ].InvMass),
    acopla = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_2 ].Acopl),   
    extratracks = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_2 ].ExtraPfCands), 
    weight = np.array(MonteCarlo_SemiExclusive[ msk_sgsex_pred_2 ].Weight*Norm_Test)
    )
histo.fill(
    sample = "Data-2017",
    pt = np.array(df_data[msk_data_2].pt),
    massa_inv = np.array(df_data[msk_data_2].InvMass),
    acopla = np.array(df_data[msk_data_2].Acopl),  
    extratracks = np.array(df_data[ msk_data_2].ExtraPfCands) 
    )

plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
})

fill_opts = {
    'edgecolor': (0,0,0,0.3),
    'alpha': 0.8
}
error_opts = {
    'label': 'Stat. Unc.',
    'hatch': '///',
    'facecolor': 'none',
    'edgecolor': (0,0,0,1),
    'linewidth': 0
}
data_err_opts = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1,
}

import re
notdata = re.compile('(?!Data-2017)')

In [ ]:
fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True
)
fig.subplots_adjust(hspace=.07)

pt1hist = histo.sum("extratracks", "massa_inv", 'acopla')

hist.plot1d(
    pt1hist['Data-2017'],
    overlay="sample",
    clear=True,
    ax=ax,
    error_opts=data_err_opts
    )
hist.plot1d(
    pt1hist[notdata],
    overlay='sample',
    stack=True,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],
    clear=False,
    ax=ax,
    line_opts=False,
    fill_opts=fill_opts,
    error_opts=error_opts
    )

#plt.yscale('log')
#plt.style.use([hep.style.ROOT, hep.style.firamath])
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 4)
ax.set_xlabel(None)
leg = ax.legend()

hist.plotratio(
    num=pt1hist['Data-2017'].sum("sample"),
    denom=pt1hist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_pt_normalized.png')
plt.close()

In [ ]:

masshist = histo.sum("pt", "extratracks", 'acopla')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    masshist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    masshist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.ylim(0, 1750)
#plt.yscale('log')
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 5)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=masshist['Data-2017'].sum("sample"),
    denom=masshist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

In [ ]:
extrahist = histo.sum("pt", "massa_inv", 'acopla')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    extrahist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    extrahist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.ylim(0, 1750)
#plt.yscale('log')
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 16)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=extrahist['Data-2017'].sum("sample"),
    denom=extrahist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

In [ ]:
acoplahist = histo.sum("pt", "massa_inv", 'extratracks')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    acoplahist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    acoplahist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 5)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=acoplahist['Data-2017'].sum("sample"),
    denom=acoplahist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

# **SOMA DOS HISTOGRAMAS**

In [ ]:
histo = hist.Hist("Events",
        hist.Cat("sample", ""),
        hist.Bin("pt", "$P_{T_{\mu^{+}\mu^{-}}}(GeV/c)$", 15, 0, 10),
        hist.Bin("massa_inv", "$M_{\mu^{+}\mu^{-}}(GeV/c^{2})$", 15, 0, 500),
        hist.Bin("acopla", "Acoplanaridade", 20, 0, 0.004),   
        hist.Bin("extratracks", "Traços Extras", 20, 0, 40)
        )
histo.fill(
    sample = "Data-driven Background",
    pt = np.array(MonteCarlo_DrellYan.pt),
    massa_inv = np.array(MonteCarlo_DrellYan.InvMass),
    acopla = np.array(MonteCarlo_DrellYan.Acopl),
    extratracks = np.array(MonteCarlo_DrellYan.ExtraPfCands),
    weight = np.array(MonteCarlo_DrellYan.Weight*Norm_Test)
    )
histo.fill(
    sample = "Exclusivo",
    pt = np.array(MonteCarlo_Exclusive.pt),
    massa_inv = np.array(MonteCarlo_Exclusive.InvMass),
    acopla = np.array(MonteCarlo_Exclusive.Acopl),   
    extratracks = np.array(MonteCarlo_Exclusive.ExtraPfCands), 
    weight = np.array(MonteCarlo_Exclusive.Weight*Norm_Test)
    )
histo.fill(
    sample = "Semi-Exclusivo",
    pt = np.array(MonteCarlo_SemiExclusive.pt),
    massa_inv = np.array(MonteCarlo_SemiExclusive.InvMass),
    acopla = np.array(MonteCarlo_SemiExclusive.Acopl),   
    extratracks = np.array(MonteCarlo_SemiExclusive.ExtraPfCands), 
    weight = np.array(MonteCarlo_SemiExclusive.Weight*Norm_Test)
    )
histo.fill(
    sample = "Data-2017",
    pt = np.array(df_data.pt),
    massa_inv = np.array(df_data.InvMass),
    acopla = np.array(df_data.Acopl),  
    extratracks = np.array(df_data.ExtraPfCands) 
    )

plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
})

fill_opts = {
    'edgecolor': (0,0,0,0.3),
    'alpha': 0.8
}
error_opts = {
    'label': 'Stat. Unc.',
    'hatch': '///',
    'facecolor': 'none',
    'edgecolor': (0,0,0,1),
    'linewidth': 0
}
data_err_opts = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1,
}

import re
notdata = re.compile('(?!Data-2017)')

In [ ]:
fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True
)
fig.subplots_adjust(hspace=.07)

pt1hist = histo.sum("extratracks", "massa_inv", 'acopla')

hist.plot1d(
    pt1hist['Data-2017'],
    overlay="sample",
    clear=True,
    ax=ax,
    error_opts=data_err_opts
    )
hist.plot1d(
    pt1hist[notdata],
    overlay='sample',
    stack=True,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],
    clear=False,
    ax=ax,
    line_opts=False,
    fill_opts=fill_opts,
    error_opts=error_opts
    )

#plt.yscale('log')
#plt.style.use([hep.style.ROOT, hep.style.firamath])
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 4)
ax.set_xlabel(None)
leg = ax.legend()

hist.plotratio(
    num=pt1hist['Data-2017'].sum("sample"),
    denom=pt1hist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_pt_normalized.png')
plt.close()

In [ ]:
acoplahist = histo.sum("pt", "massa_inv", 'extratracks')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    acoplahist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    acoplahist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 5)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=acoplahist['Data-2017'].sum("sample"),
    denom=acoplahist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

In [ ]:
extrahist = histo.sum("pt", "massa_inv", 'acopla')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    extrahist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    extrahist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.ylim(0, 1750)
#plt.yscale('log')
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 16)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=extrahist['Data-2017'].sum("sample"),
    denom=extrahist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()

In [ ]:

masshist = histo.sum("pt", "extratracks", 'acopla')

fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(10,10),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True

)
fig.subplots_adjust(hspace=.07)

hist.plot1d(
    masshist[notdata],
    overlay='sample',
    stack=True,
    ax=ax,
    order = ['Exclusivo', "Semi-Exclusivo", "Data-driven Background"],    
    line_opts=None,
    fill_opts=fill_opts,
    error_opts=error_opts 
)
hist.plot1d(
    masshist['Data-2017'],
    overlay="sample",
    ax=ax,
    clear=False,
    error_opts=data_err_opts,
    density = False
)
plt.ylim(0, 1750)
#plt.yscale('log')
plt.legend(loc="upper right")
ax.autoscale(axis='x', tight=True)
ax.set_ylim(0, 5)
ax.set_xlabel(None)
#ax = hep.cms.cmslabel(data=False, paper=False, year='$18.34 fb^{-1}$', ax=ax)
leg = ax.legend()

hist.plotratio(
    num=masshist['Data-2017'].sum("sample"),
    denom=masshist[notdata].sum("sample"),
    ax=rax,
    error_opts=data_err_opts,
    denom_fill_opts={},
    guide_opts={},
    unc='num'
)
rax.set_ylabel('Ratio')
rax.set_ylim(0,2)
plt.tight_layout()
plt.show()
#plt.savefig('/home/matheus/test/hist_massa_normalized.png')
plt.close()